src : https://blog.langchain.dev/reflection-agents/

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph
model = ChatOllama(model="llama3.2",
                    temperature=0,)

In [3]:


reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



generate_chain = generation_prompt | model
reflect_chain = reflection_prompt | model




REFLECT = "reflect"
GENERATE = "generate"

In [4]:
def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]


def should_continue(state: List[BaseMessage]):
    #how many times it should run before it stop
    if len(state) > 3:
        return END
    return REFLECT


builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)
builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)
graph = builder.compile()
graph.get_graph().draw_mermaid_png(output_file_path = "reflection_agent.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xe4\x00\x00\x00\xf9\x08\x02\x00\x00\x00\xb8}\x0e\x9f\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x9c\xed\x9di\\S\xc7\xfe\xc6\'\x0b!;\x81$\x84% \x08\x88(\xa0(VE\xd0\xe2RZw\xabE[\xb1\xb6\xd5Z\xbc.\xf5V[i\xc1[\xb5\xd5*\xb6\xd6\xbf\x0b\xb6W\xabVQ\xaf\xb5\xd6\xaa\xd5\xba\xd4}C\x10\x97\xb2\x15D\x91}M !\xfb\xfa\x7f\x11?\x94j@"I\xe6\x9cd\xbe\xaf\x92\x93\xc9\x99\'9O&3s~\xf3\x1b\x82\xd1h\x04\x08\x04\x1e \xc2\x16\x80@t\x15dV\x04n@fE\xe0\x06dV\x04n@fE\xe0\x06dV\x04n \xc3\x16`\x13j\x1f+\x15R\xbd\xa2U\xaf\xd7\x195*\x03l9]\x82B%\xba\xd2\x88\x0c6\x99\xce&\xf1|\\a\xcb\xc1"\x04\x87\x99g5\x1a\x8dE\xd9\xad\x8f\xf2d\x8f\x0b\x14\xfe\xbd\xe9d\x17\x02\x9dE\xe2xR4J|\x98\x95@\x04\x92&\xad\\\xaa\xa31H5\x8fT\x81\xe1\x8c\xa0\x08\x86\xb0\x17\x1d\xb6.\x0c\xe1 f\xbd{\xb1\xf9\xee\xc5\xe6\x1ea\x8c\x9e\x11\xcc\xc0p\x06l9\xdd\xa5\xb5Y[\x96/o\xa8R\xb7\xd4k\x87N\xe0\xfa\x06\xd1`+\xc2\x04\xb87ke\x89\xe2\xcc\x8fu\xbd_b\x0f\x9b\xc0%\x10\t\xb0\xe5X\x99\xda\xc7\xca

In [5]:
inputs = HumanMessage(content="""Make this tweet better:"
                                    @LangChainAI
            — newly Tool Calling feature is seriously underrated.

            After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

            Made a video covering their newest blog post

                                  """)
response = graph.invoke(inputs, debug=True)

[-1:checkpoint] State at the end of step -1:
{'__root__': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ', additional_kwargs={}, response_metadata={})
[0:writes] Finished step 0 with writes to 1 channel:
- __root__ -> HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\

In [7]:
response

[HumanMessage(content='Make this tweet better:"\n                                    @LangChainAI\n            — newly Tool Calling feature is seriously underrated.\n\n            After a long wait, it\'s  here- making the implementation of agents across different models with function calling - super easy.\n\n            Made a video covering their newest blog post\n\n                                  ', additional_kwargs={}, response_metadata={}, id='c145b155-f080-4b4c-b24d-ee302c98aae8'),
 AIMessage(content='Here\'s a rewritten version of the tweet:\n\n"Exciting news! @LangChainAI just dropped the \'Tool Calling\' feature, and I\'m obsessed! This game-changing tool makes it ridiculously easy to implement agents across different models with function calling. Check out my latest video review on their newest blog post [link] #LangChainAI #ToolCalling"\n\nI made a few changes to improve the tweet:\n\n* Added a more attention-grabbing opening line\n* Emphasized the benefits of the Tool Ca